Algorithm: Automatic Panorama Stitching

Input: n unordered images

I. Extract SIFT features from all n images

II. Find k nearest-neighbours for each feature using a k-d tree

III. For each image:

(i) Select m candidate matching images that have
    the most feature matches to this image

(ii) Find geometrically consistent feature matches
    using RANSAC to solve for the homography between
    pairs of images

(iii) Verify image matches using a probabilistic model

IV. Find connected components of image matches

V. For each connected component:

(i) Perform bundle adjustment to solve for the rotation
    1, 2, 3 and focal length f of all cameras

(ii) Render panorama using multi-band blending
Output: Panoramic image(s)

In [1]:
import numpy as np
import cv2

Function that extracts SIFT features

In [2]:
def sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    descriptors = cv2.xfeatures2d.SIFT_create()
    (keypoints, features) = descriptors.detectAndCompute(gray, None)
    
    keypoints = np.float32([i.pt for i in keypoints])
        
    return (keypoints, features)

Verifies image matches using a probabilistic model

Finds connected components of image matches

In [43]:
def match_keypoints(keypoints1, keypoints2, features1, features2, lowe_ratio):
    
    # first find all possible matches between features
    all_matches = all_possible_matches(features1, features2)
    # then from all possible matches select valid matched according to the paper(Dawid Lowe concept)
    valid_matches = all_valid_matches(all_matches, lowe_ratio)

    # According to the paper we chose 4 matches to be enough for matching to take place
    if len(valid_matches) > 4:
        points1 = np.float32([keypoints1[i] for (_,i) in valid_matches])
        points2 = np.float32([keypoints2[i] for (i,_) in valid_matches])
        
        # We compute transformation matrix needed to transform perspective of first picture
        homography = compute_homography(points1, points2)

        return homography
    else:
        return None

def all_possible_matches(features1, features2):
    match_instance = cv2.DescriptorMatcher_create("BruteForce")
    all_matches = match_instance.knnMatch(features1, features2, 2)

    return all_matches

def all_valid_matches(all_matches, lowe_ratio):
    valid_matches = []

    for val in all_matches:
        if len(val) == 2 and val[0].distance < val[1].distance * lowe_ratio:
            valid_matches.append((val[0].trainIdx, val[0].queryIdx))

    return valid_matches

Function finds geometrically consistent feature matches using RANSAC to solve for the homography between pairs of images

In [44]:
def compute_homography(points1, points2):
    (homography, status) = cv2.findHomography(points1, points2, cv2.RANSAC, 0.4)

    return homography

 Perform bundle adjustment to solve for the rotation 1, 2, 3 and focal length f of all cameras

In [29]:

def get_warp_perspective(train_img, query_img, homography):
    t_row , t_col = train_img.shape[: 2]
    q_row , q_col = query_img.shape[: 2]
    
    list_pts1 = np.float32([[0 , 0] , [0 , t_row] , [t_col , t_row] , [t_col , 0]]).reshape(-1 , 1 , 2)
    temp_points = np.float32([[0 , 0] , [0 , q_row] , [q_col , q_row] , [q_col , 0]]).reshape(-1 , 1 , 2)

    list_pts2 = cv2.perspectiveTransform(temp_points , homography)

    list_pts = np.concatenate((list_pts1 , list_pts2) , axis = 0)   

    [x_min , y_min] = np.int32(list_pts.min(axis = 0).ravel() - 0.5)
    [x_max , y_max] = np.int32(list_pts.max(axis = 0).ravel() + 0.5)

    translation_dist = [-x_min , -y_min]
    #translation_dist = [0, 0]
    
    #x = train_img.shape[1] + query_img.shape[1]

    homography_translation = np.array([[1 , 0 , translation_dist[0]] , [0 , 1 , translation_dist[1]] , [0 , 0 , 1]])
    
    #warped = cv2.warpPerspective(query_img , homography_translation.dot(homography) , (x, int(train_img.shape[0])))
    warped = cv2.warpPerspective(query_img , homography_translation.dot(homography) , (x_max - x_min , y_max - y_min))
    warped = alpha_blend(warped, train_img, translation_dist)
    
    #warped[translation_dist[1] : t_row + translation_dist[1] , translation_dist[0] : translation_dist[0] + t_col ] = train_img
    #x = image1.shape[1] + image2.shape[1]
    #warped = cv2.warpPerspective(image1, homography, (x , int(image1.shape[0])))
    #warped = cv2.perspectiveTransform(image1, homography)
    
    return warped

Function that renders panorama

In [26]:
def stitch(images):
    (image2, image1) = images
    
    panorama[0:image2.shape[0], 0:image2.shape[1]] = image2
    
    return panorama

In [27]:
def alpha_blend(img1, img2, translation_dist):
    
    t_row , t_col = img2.shape[: 2]
    # I want to put logo on top-left corner, So I create a ROI
    rows, cols, channels = img2.shape
    
    roi = img1[translation_dist[1] : t_row + translation_dist[1] , translation_dist[0] : translation_dist[0] + t_col]
    
    # Now create a mask of logo and create its inverse mask also
    img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)
    
    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)
    
    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(img2, img2, mask = mask)
    
    # Put logo in ROI and modify the main image
    dst = cv2.add(img1_bg, img2_fg)
    img1[translation_dist[1] : t_row + translation_dist[1] , translation_dist[0] : translation_dist[0] + t_col ] = dst

    return img1

In [42]:
lowe_ratio=0.75

images = []

images.append(cv2.imread('coala1.png'))
images.append(cv2.imread('coala2.png'))

# Extract SIFT features from all images
keypoints = []
features = []
for image in images:
    (keypoint, feature) = sift_features(image)

    keypoints.append(keypoint)
    features.append(feature)

# Compute all pairwise homographies
homographies = {} 
for i in range(len(images)):
    for j in range(i+1, len(images)):
        # image that is being warped is image j and we send it first
        homography = match_keypoints(keypoints[j], keypoints[i], features[j], features[i], lowe_ratio)
        homographies[(i, j)] = homography

homography = np.identity(3)
ind = True
for i in range(len(images)-1):
    homography = homographies[(i, i+1)]
    #homography = homography @ homographies[(i, i+1)]
    if i==0:
        #homography = homography @ homographies[(i, i+1)]
        panorama = get_warp_perspective(images[i], images[i+1], homography)
    else:
        (keypoints_p, features_p) = sift_features(panorama)
        homography = match_keypoints(keypoints[i+1], keypoints_p, features[i+1], features_p, lowe_ratio)
        panorama = get_warp_perspective(panorama, images[i+1], homography)
    
cv2.imwrite("panorama.jpg", panorama)

True